<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in Toronto. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
# import folium # map rendering library

# print('Libraries imported.')

ImportError: No module named 'folium'

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [3]:
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

The Toronto neighborhood file was located here.  https://gis-kingcounty.opendata.arcgis.com/datasets/neighborhood-centers-in-king-county-neighborhood-centers-point

In [4]:
!wget -q -O 'toronto_data.csv' https://opendata.arcgis.com/datasets/d3943c8a3af743e5a27abb44687fc957_385.csv
print('Data downloaded!')

Data downloaded!


In [5]:
neighborhoods = pd.read_csv('toronto_data.csv')
neighborhoods.head()

,X,Y,OBJECTID,FEATURE_ID,CODE,NAME,ADDRESS,NEIGH_NUM,XCOORD,YCOORD,LONGITUDE,LATITUDE
0,-122.355548,47.723233,1,90001,Community Area/Business Center/Neighborhood,Bitter Lake Neighborhood,N 130th St & Greenwood Ave N,3,1.265776e+06,267518.406033,-122.355550,47.723232
1,-122.323274,47.708595,2,90002,Community Area/Business Center/Neighborhood,Northgate Neighborhood,NE Northgate Way & 5th Ave NE,28,1.273615e+06,262024.468858,-122.323276,47.708593
2,-122.295226,47.719280,3,90003,Community Area/Business Center/Neighborhood,Lake City Neighborhood,NE 125th & Lake City Way NE,21,1.280593e+06,265788.625044,-122.295228,47.719278
3,-122.229209,47.727482,4,90004,Community Area/Business Center/Neighborhood,Finn Hill Neighborhood,NE 141st St & 84th Ave NE,13,1.296895e+06,268478.406082,-122.229210,47.727480
4,-122.290272,47.675785,5,90005,Community Area/Business Center/Neighborhood,Wedgewood Neighborhood,NE 65th St & 35th Ave NE,39,1.281512e+06,249903.141067,-122.290273,47.675783


In [6]:
neighborhoods.drop(['X','Y','OBJECTID','FEATURE_ID','CODE','ADDRESS','XCOORD','YCOORD'], axis = 1,inplace=True)
neighborhoods.head()

,NAME,NEIGH_NUM,LONGITUDE,LATITUDE
0,Bitter Lake Neighborhood,3,-122.355550,47.723232
1,Northgate Neighborhood,28,-122.323276,47.708593
2,Lake City Neighborhood,21,-122.295228,47.719278
3,Finn Hill Neighborhood,13,-122.229210,47.727480
4,Wedgewood Neighborhood,39,-122.290273,47.675783


In [7]:
print('The dataframe has {} NAMES and {} neighborhoods.'.format(
        len(neighborhoods['NAME'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 46 NAMES and 46 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>YYZ_explorer</em>, as shown below.

In [8]:
address = 'Toronto,WA'

geolocator = Nominatim(user_agent="YYZ_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 47.6038321, -122.3300624.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neigh_num, neighborhood in zip(neighborhoods['LATITUDE'], neighborhoods['LONGITUDE'], neighborhoods['NEIGH_NUM'], neighborhoods['NAME']):
    label = '{}, {}'.format(neighborhood, neigh_num)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'IJ5W52TJAI3JAMNFGMYAMDCW0NP2MGKG240YVJYI4EQ2F12I' # your Foursquare ID
CLIENT_SECRET = 'VSVBTV3D4VIROFHDAZFJPQHC303OBSBLFTQZNMU030GCPXPV' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IJ5W52TJAI3JAMNFGMYAMDCW0NP2MGKG240YVJYI4EQ2F12I
CLIENT_SECRET:VSVBTV3D4VIROFHDAZFJPQHC303OBSBLFTQZNMU030GCPXPV


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [11]:
neighborhoods.loc[0, 'NAME']

'Bitter Lake Neighborhood'

Get the neighborhood's latitude and longitude values.

In [12]:
neighborhood_latitude = neighborhoods.loc[0, 'LATITUDE'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'LONGITUDE'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'NAME'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bitter Lake Neighborhood are 47.72323154, -122.35554967.


#### Now, let's get the top 100 venues that are in Bitter Lake within a radius of 500 meters.

In [19]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 800 # define radius

First, let's create the GET request URL. Name your URL **url**.

In [20]:
# type your answer here
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=IJ5W52TJAI3JAMNFGMYAMDCW0NP2MGKG240YVJYI4EQ2F12I&client_secret=VSVBTV3D4VIROFHDAZFJPQHC303OBSBLFTQZNMU030GCPXPV&v=20180604&ll=47.72323154,-122.35554967&radius=800&limit=1000'

Send the GET request and examine the resutls

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd6257fdb04f559dbfa8413'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-56f9f766498e4e8a7b51dbdd-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1cb941735',
         'name': 'Food Truck',
         'pluralName': 'Food Trucks',
         'primary': True,
         'shortName': 'Food Truck'}],
       'id': '56f9f766498e4e8a7b51dbdd',
       'location': {'address': 'Location varies',
        'cc': 'US',
        'city': 'Toronto',
        'country': 'United States',
        'distance': 394,
        'formattedAddress': ['Location varies',
         'Toronto, WA 98109',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 47

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wicked Good Grinders,Food Truck,47.719704,-122.356108
1,Chada Thai,Thai Restaurant,47.719663,-122.355114
2,JoyFish,Szechuan Restaurant,47.724722,-122.345950
3,Style Hot Pot,Hotpot Restaurant,47.723546,-122.347021
4,Planet Fitness,Gym / Fitness Center,47.725857,-122.346222


And how many venues were returned by Foursquare?

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [65]:
# type your answer here

toronto_venues = getNearbyVenues(names=neighborhoods['NAME'],
                                   latitudes=neighborhoods['LATITUDE'],
                                   longitudes=neighborhoods['LONGITUDE']
                                  )



Bitter Lake Neighborhood
Northgate Neighborhood
Lake City Neighborhood
Finn Hill Neighborhood
Wedgewood Neighborhood
University District
Green Lake Neighborhood
Greenwood Neighborhood
North Beach Neighborhood
Crown Hill Neighborhood
Ballard Neighborhood
Fremont Neighborhood
Wallingford Neighborhood
Queen Anne Neighborhood
Capitol Hill Neighborhood
Central District
Central Business District
Harbor Island Neighborhood
West Toronto Neighborhood
Beacon Hill Neighborhood
Mount Baker Neighborhood
Sand Point Neighborhood
Seward Park Neighborhood
Bryn Mawr Neighborhood
Skyway Neighborhood
South Park Neighborhood
White Center Neighborhood
Eastgate Neighborhood
Rainier Valley Neighborhood
Maple Leaf Neighborhood
Twin Lakes Neighborhood
Magnolia Neighborhood
Rainier Beach Neighborhood
Covington, Timberlane Neighborhood
North Creek Neighborhood
Fairwood Neighborhood
Georgetown Neighborhood
Renton Highlands Neighborhood
Kent East Hill Neighborhood
Kingsgate Neighborhood
Newport Hills Neighborhood
C

#### Let's check the size of the resulting dataframe

In [66]:
print(toronto_venues.shape)
toronto_venues.head()

(853, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bitter Lake Neighborhood,47.723232,-122.355550,Veridian Cove Pool,47.723852,-122.353542,Pool
1,Bitter Lake Neighborhood,47.723232,-122.355550,Veridian Cove Gym,47.723854,-122.353419,Gym
2,Northgate Neighborhood,47.708593,-122.323276,Total Wine & More,47.708358,-122.321576,Liquor Store
3,Northgate Neighborhood,47.708593,-122.323276,24 Hour Fitness,47.708107,-122.322798,Gym / Fitness Center
4,Northgate Neighborhood,47.708593,-122.323276,Jimmy John's,47.708551,-122.322508,Sandwich Place


Let's check how many venues were returned for each neighborhood

In [67]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Admiral Neighborhood,35,35,35,35,35,35
Alki Neighborhood,18,18,18,18,18,18
Ballard Neighborhood,47,47,47,47,47,47
Beacon Hill Neighborhood,12,12,12,12,12,12
Bitter Lake Neighborhood,2,2,2,2,2,2
Bryn Mawr Neighborhood,2,2,2,2,2,2
Capitol Hill Neighborhood,50,50,50,50,50,50
Central Business District,41,41,41,41,41,41
Central District,14,14,14,14,14,14


#### Let's find out how many unique categories can be curated from all the returned venues

In [68]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 197 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [69]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,ATM,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dumpling Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Fondue Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Knitting Store,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bitter Lake Neighborhood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bitter Lake Neighborhood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Northgate Neighborhood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [70]:
toronto_onehot.shape

(853, 197)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [71]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dumpling Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Fondue Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Knitting Store,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store
0,Admiral Neighborhood,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.028571,0.028571,0.057143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.057143,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.028571,0.000000,0.028571,0.000000,0.000000,0.00000,0.000000,0.000000,0.028571,0.000000,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.057143,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,0.028571,0.000000,

#### Let's confirm the new size

In [72]:
toronto_grouped.shape

(44, 197)

#### Let's print each neighborhood along with the top 5 most common venues

In [73]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Admiral Neighborhood----
           venue  freq
0    Pizza Place  0.09
1            Bar  0.06
2            Pub  0.06
3  Grocery Store  0.06
4    Coffee Shop  0.06


----Alki Neighborhood----
              venue  freq
0       Coffee Shop  0.17
1        Sports Bar  0.06
2    Sandwich Place  0.06
3  Sushi Restaurant  0.06
4    History Museum  0.06


----Ballard Neighborhood----
                venue  freq
0  Mexican Restaurant  0.06
1         Coffee Shop  0.06
2        Dessert Shop  0.04
3                 Pub  0.04
4      Clothing Store  0.04


----Beacon Hill Neighborhood----
                venue  freq
0  Dim Sum Restaurant  0.08
1         Coffee Shop  0.08
2          Steakhouse  0.08
3         Supermarket  0.08
4       Grocery Store  0.08


----Bitter Lake Neighborhood----
                   venue  freq
0                   Pool   0.5
1                    Gym   0.5
2            Yoga Studio   0.0
3  Performing Arts Venue   0.0
4              Nightclub   0.0


----Bryn Mawr Neighborho

                   venue  freq
0            Pizza Place   1.0
1            Yoga Studio   0.0
2  Performing Arts Venue   0.0
3              Nightclub   0.0
4           Noodle House   0.0


----West Toronto Neighborhood----
                venue  freq
0         Pizza Place  0.09
1  Mexican Restaurant  0.07
2              Bakery  0.05
3         Coffee Shop  0.05
4  Falafel Restaurant  0.02


----White Center Neighborhood----
         venue  freq
0     Pharmacy  0.10
1  Coffee Shop  0.10
2  Roller Rink  0.05
3    BBQ Joint  0.05
4  Pizza Place  0.05




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [74]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiral Neighborhood,Pizza Place,Bar,Pub,Pet Store,Grocery Store,Coffee Shop,Sushi Restaurant,Salon / Barbershop,Gas Station,Frozen Yogurt Shop
1,Alki Neighborhood,Coffee Shop,Pizza Place,Sushi Restaurant,Café,Donut Shop,Fish & Chips Shop,Outdoor Sculpture,Sandwich Place,Sports Bar,Seafood Restaurant
2,Ballard Neighborhood,Mexican Restaurant,Coffee Shop,Clothing Store,Burger Joint,Pub,Sushi Restaurant,Thai Restaurant,Dessert Shop,Vietnamese Restaurant,Latin American Restaurant
3,Beacon Hill Neighborhood,Pizza Place,Gym / Fitness Center,Garden Center,Gas Station,Steakhouse,Supermarket,Bar,Bakery,BBQ Joint,Grocery Store
4,Bitter Lake Neighborhood,Pool,Gym,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Event Space


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [77]:
# set number of clusters
kclusters = 25

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([22,  1,  3, 24,  7,  8, 22, 22, 21, 22], dtype=int32)

In [78]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiral Neighborhood,Pizza Place,Bar,Pub,Pet Store,Grocery Store,Coffee Shop,Sushi Restaurant,Salon / Barbershop,Gas Station,Frozen Yogurt Shop
1,Alki Neighborhood,Coffee Shop,Pizza Place,Sushi Restaurant,Café,Donut Shop,Fish & Chips Shop,Outdoor Sculpture,Sandwich Place,Sports Bar,Seafood Restaurant
2,Ballard Neighborhood,Mexican Restaurant,Coffee Shop,Clothing Store,Burger Joint,Pub,Sushi Restaurant,Thai Restaurant,Dessert Shop,Vietnamese Restaurant,Latin American Restaurant
3,Beacon Hill Neighborhood,Pizza Place,Gym / Fitness Center,Garden Center,Gas Station,Steakhouse,Supermarket,Bar,Bakery,BBQ Joint,Grocery Store
4,Bitter Lake Neighborhood,Pool,Gym,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Event Space


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='NAME')

toronto_merged.head() # check the last columns!

,NAME,NEIGH_NUM,LONGITUDE,LATITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bitter Lake Neighborhood,3,-122.355550,47.723232,7.0,Pool,Gym,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop,Event Space
1,Northgate Neighborhood,28,-122.323276,47.708593,22.0,Coffee Shop,Pizza Place,Dance Studio,Sandwich Place,Mexican Restaurant,Men's Store,Sporting Goods Shop,Salon / Barbershop,Big Box Store,Liquor Store
2,Lake City Neighborhood,21,-122.295228,47.719278,3.0,Bus Station,Sushi Restaurant,Shipping Store,Bar,Thai Restaurant,Bakery,Burger Joint,Bus Stop,Mexican Restaurant,Café
3,Finn Hill Neighborhood,13,-122.229210,47.727480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wedgewood Neighborhood,39,-122.290273,47.675783,5.0,Pizza Place,Women's Store,Drugstore,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop


In [86]:
toronto_merged = toronto_merged.dropna()
toronto_merged = toronto_merged.astype({"Cluster Labels": int})

Finally, let's visualize the resulting clusters

In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['LATITUDE'], toronto_merged['LONGITUDE'], toronto_merged['NAME'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,International District,Chinese Restaurant,Japanese Restaurant,Tea Room,Bakery,Poke Place,Taiwanese Restaurant,Museum,Noodle House,Dessert Shop,Performing Arts Venue


#### Cluster 2

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Alki Neighborhood,Coffee Shop,Pizza Place,Sushi Restaurant,Café,Donut Shop,Fish & Chips Shop,Outdoor Sculpture,Sandwich Place,Sports Bar,Seafood Restaurant


#### Cluster 3

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Mount Baker Neighborhood,Park,Women's Store,Food Truck,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop


#### Cluster 4

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Lake City Neighborhood,Bus Station,Sushi Restaurant,Shipping Store,Bar,Thai Restaurant,Bakery,Burger Joint,Bus Stop,Mexican Restaurant,Café
5,University District,Chinese Restaurant,Thrift / Vintage Store,Korean Restaurant,Bubble Tea Shop,Bar,Tea Room,Mediterranean Restaurant,Bank,Greek Restaurant,Record Shop
6,Green Lake Neighborhood,Coffee Shop,Pharmacy,Frozen Yogurt Shop,Mexican Restaurant,Thai Restaurant,Women's Store,Karaoke Bar,Shoe Store,Sandwich Place,Playground
7,Greenwood Neighborhood,Bar,Coffee Shop,Theater,Mexican Restaurant,Spa,Bookstore,Vegetarian / Vegan Restaurant,Sports Bar,Diner,Pharmacy
10,Ballard Neighborhood,Mexican Restaurant,Coffee Shop,Clothing Store,Burger Joint,Pub,Sushi Restaurant,Thai Restaurant,Dessert Shop,Vietnamese Restaurant,Latin American Restaurant
12,Wallingford Neighborhood,Pharmacy,Japanese Restaurant,Coffee Shop,Pub,Pet Store,Bank,Bridal Shop,Shopping Mall,Dive Bar,Lounge
13,Queen Anne Neighborhood,Coffee Shop,Shipping Store,Thai Restaurant,Lounge,Pharmacy,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,Pet Store,Sandwich Place
26,White Center Neighborhood,Coffee Shop,Pharmacy,Burger Joint,Mexican Restaurant,Gay Bar,Bar,Bakery,BBQ Joint,Asian Restaurant,Lounge
36,Georgetown Neighborhood,French Restaurant,Coffee Shop,Pizza Place,Bookstore,Café,Mediterranean Restaurant,Sandwich Place,Bar,Dive Bar,Brewery
38,Kent East Hill Neighborhood,Mexican Restaurant,Japanese Restaurant,Pharmacy,Bar,Ice Cream Shop,Coffee Shop,Mobile Phone Shop,Sandwich Place,Salon / Barbershop,Fried Chicken Joint


#### Cluster 5

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,North Beach Neighborhood,Bus Stop,Women's Store,French Restaurant,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).